In [ ]:
using PowerModels
using Ipopt
using ForwardDiff
using OrdinaryDiffEq
using NLsolve

display(HTML("<style>.container { width:100% !important; }</style>"))

data = PowerModels.parse_file("C:/Users/Ewa/Downloads/Twogen2bus.m")
pf_result = run_ac_pf(data ,Ipopt.Optimizer)
#PowerModels.print_summary(pf_result["solution"])

# Load flow results

# bus results
va1 =pf_result["solution"]["bus"]["1"]["va"]
va2 =pf_result["solution"]["bus"]["2"]["va"]
vm1 =pf_result["solution"]["bus"]["1"]["vm"]
vm2 =pf_result["solution"]["bus"]["2"]["vm"];

# Voltage at bus 1 (grid forming)
V_tR = vm1*cos(va1)
V_tI = vm1*sin(va1)


# Voltage at bus 2 (grid following)
V_tR1 = vm2*cos(va2)
V_tI1 = vm2*sin(va2)

# generator results
pg = pf_result["solution"]["gen"]["1"]["pg"]
qg =pf_result["solution"]["gen"]["1"]["qg"]

# branch results
p_ref =pf_result["solution"]["branch"]["1"]["pf"]
q_ref =pf_result["solution"]["branch"]["1"]["qf"]
p_ref1 =pf_result["solution"]["branch"]["1"]["pt"]
q_ref1 =pf_result["solution"]["branch"]["1"]["qt"];

In [ ]:
PowerModels.print_summary(pf_result["solution"])

In [ ]:
# Define the parameters
lf =0.08
rf = 0.03
cf = 0.074

# Need to find values for these

rt =0.00005
lt = 0.15
rl = 1.0

Rp = 2
Rq =0.1
ωz = 2*pi*5.0


kvp = 0.52
kvi = 1.16
kvf = 1.0
rv = 0.0
lv =0.2
kip = 0.74
kii = 1.19
kif = 1.0

ωg = 1.0 #ω_apc 
ω_ref = 1.0
v_ref = vm1
ωb = 2*pi*50.0
ω_star = ωg


In [ ]:
function system!(F,x)
    
    ω_apc = ω_star + Rp*(p_ref - x[7])
    v_hat = v_ref + Rq*(q_ref - x[8])
    v_iref_d = v_hat - rv*x[5] + ω_apc*lv*x[6]
    v_iref_q = - rv*x[6] - ω_apc*lv*x[5]
    i_hat_d = kvp*(v_iref_d -x[1]) + kvi*x[10] - ω_apc*cf*x[2] + kif*x[5]
    i_hat_q = kvp*(v_iref_d -x[2]) + kvi*x[11] + ω_apc*cf*x[1] + kif*x[6]
    v_md = kip*(i_hat_d -x[3]) + kii*x[12] - ω_apc*lf*x[4] + kvf*x[1]
    v_mq = kip*(i_hat_q -x[4]) + kii*x[13] + ω_apc*lf*x[3] + kvf*x[2]
   # v_gd = rl*x[5]
    #v_gq = rl*x[6]
    v_gd = V_tR
    v_gq = V_tI
    
    
### Grid forming equations
  #𝜕eg/𝜕t
F[1]= (ωb / cf)*(x[3] - x[5]) + ωg*ωb*x[2]
  #𝜕eq/𝜕t
F[2]= (ωb / cf)*(x[4] - x[6]) - ωg*ωb*x[1]
   #𝜕isd/𝜕t
F[3]= (ωb / lf)*(v_md - x[1]) - (rf*ωb/lf)*x[3] + ωb*ωg*x[4]
   #𝜕isq/𝜕t
F[4]= (ωb / lf)*(v_mq - x[2]) - (rf*ωb/lf)*x[4] - ωb*ωg*x[3]
     #𝜕igd/𝜕t
F[5]= (ωb / lt)*(x[1] - v_gd) - (rt*ωb/lt)*x[5] + ωb*ωg*x[6]
   #𝜕igq/𝜕t
F[6]= (ωb / lt)*(x[2] - v_gq) - (rt*ωb/lt)*x[6] - ωb*ωg*x[5] 
   #𝜕pm/𝜕t
F[7]=  (ωz*x[6]*x[2] + ωz*x[5]*x[1]- ωz*x[7])
   #𝜕qm/𝜕t
F[8]= (-ωz*x[6]*x[1] + ωz*x[5]*x[2] -ωz*x[8])
  #𝜕δc/𝜕t
F[9]= ωb*ω_apc
   #𝜕ξ_d/𝜕t
F[10]= v_iref_d - x[1]
   #𝜕ξ_q/𝜕t
F[11]= v_iref_q - x[2]
   #𝜕γ_d/𝜕t
F[12]=i_hat_d - x[3]
    #𝜕γ_q/𝜕t
F[13]= i_hat_q - x[4]
end


# Initial Conditions
     initial =  [
           V_tR, #egd1
           V_tI, #egq1
            p_ref/V_tR, #isd1
           #-q_ref1/V_tI1, #isq
            0.0, #isq1
            p_ref/V_tR, #igd1
            0.0,#igq1
           #-q_ref1/V_tI1, #isq
            p_ref, #pm1
            q_ref, #qm1
            va1,#δc1
            0.0, #ξ_d1
            0.0, #ξ_q1
            0.0, #γ_d1
            0.0, #γ_q1
         ]


         sys_solve = NLsolve.nlsolve(
                     system!,
                     initial,
                    xtol = :1e-9,
                     ftol = :1e-9,
                     method = :trust_region,
                    ) #Solve using initial guess x0
                      print(sys_solve)